# import data

In [1]:
import pandas as pd
import numpy as np
from WindPy import *
#Wind API 使用W账户密码登录
w.start() 
#return True and then can go to next operation.
w.isconnected()

22.11.1.52440
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2021 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [2]:
df_origin = pd.read_excel('stocks.xlsx')
df_contract_value =  df_origin.set_index('date')[['lastUpdateTime','Unnamed: 82']].reset_index()
df_contract_value.columns = ['code','value','industry']
code_list = df_origin['date'].unique().tolist()
stk_code_list = []
for code in code_list:
    if len(str(code).split('.')) == 2:
        if str(code).split('.')[1] == 'HK' or str(code).split('.')[1] == 'SH' or str(code).split('.')[1] == 'SZ':
            if len(str(code).split('.')[0]) == 6:
                stk_code_list.append(code)
stk_code_str = ','.join(stk_code_list)
print(len(stk_code_list))
df_stk_wind =  w.wss(f"{stk_code_str}", "sec_name", "tradeDate=20221102;",usedf=True)[1]
code_list = df_origin['date'].unique().tolist()
stk_code_list = []
for code in code_list:
    if len(str(code).split('.')) == 2:
        if str(code).split('.')[1] == 'HK' or str(code).split('.')[1] == 'SH' or str(code).split('.')[1] == 'SZ':
            if len(str(code).split('.')[0]) == 6:
                stk_code_list.append(code)
stk_code_str = ','.join(stk_code_list)
print(len(stk_code_list))
df_stk_info = df_contract_value.set_index('code').loc[stk_code_list]
# pd.set_option('display.max_rows', None)
df_stk_info['name'] = df_stk_wind['SEC_NAME']
df_stk_clean = df_stk_info[df_stk_info['industry']!=0].fillna('工业').sort_values(by='industry').reset_index()
df_stk_clean = df_stk_clean[['code','name','value','industry']]
# pd.set_option('display.max_rows', None)
df_stk_clean['value'] = df_stk_clean['value'].apply(lambda x: format(x, 'f'))

tmp_str = ','.join(df_stk_clean['code'].tolist())
df_ipo_date = w.wss(tmp_str, "ipo_date", "",usedf=True)[1].reset_index()
df_stk_clean['IPO_DATE'] = df_ipo_date['IPO_DATE']
# df_stk_clean.to_excel('stock_hold.xlsx')

218
218


In [3]:
# pd.set_option('display.max_rows', None)
df_stk_clean

,code,name,value,industry,IPO_DATE
0,601006.SH,大秦铁路,1675401.000000,交通运输,2006-08-01
1,000088.SZ,盐田港,127506.000000,交通运输,1997-07-28
2,600798.SH,宁波海运,1770720.000000,交通运输,1997-04-23
3,000099.SZ,中信海直,500916.000000,交通运输,2000-07-31
4,002027.SZ,分众传媒,7490642.000000,传媒,2004-08-04
...,...,...,...,...,...
208,002726.SZ,龙大美食,20577789.540000,食品饮料,2014-06-26
209,002626.SZ,金达威,10064392.000000,食品饮料,2011-10-28
210,000858.SZ,五粮液,1425840.000000,食品饮料,1998-04-27
211,600197.SH,伊力特,2649348.000000,食品饮料,1999-09-16


In [4]:
idsty_stk_dic = {}
for index,row in df_stk_clean.iterrows():
    tmp_code = row['code']
    tmp_industry = row['industry']
    if tmp_industry not in idsty_stk_dic:
        idsty_stk_dic[tmp_industry] = [tmp_code]
    else:
        idsty_stk_dic[tmp_industry].append(tmp_code)
print(idsty_stk_dic.keys())
df_industry_count = pd.DataFrame(df_stk_clean['industry'].value_counts())
df_industry_count = df_industry_count.reset_index()
df_industry_count

dict_keys(['交通运输', '传媒', '公用事业', '农林牧渔', '医药生物', '商贸零售', '国防军工', '基础化工', '家用电器', '工业', '建筑材料', '建筑装饰', '房地产', '有色金属', '机械设备', '汽车', '煤炭', '环保', '电力设备', '电子', '石油石化', '社会服务', '美容护理', '计算机', '轻工制造', '通信', '钢铁', '银行', '非银金融', '食品饮料'])


,index,industry
0,医药生物,21
1,电子,21
2,电力设备,20
3,有色金属,12
4,非银金融,12
5,食品饮料,11
6,基础化工,11
7,计算机,10
8,家用电器,8
9,建筑装饰,8


# industry setting

In [5]:
target_industry = '电力设备'
df_stk_clean[df_stk_clean['industry']==target_industry]

,code,name,value,industry,IPO_DATE
122,300769.SZ,德方纳米,193935.000000,电力设备,2019-04-15
123,300763.SZ,锦浪科技,35466104.100000,电力设备,2019-03-19
124,002459.SZ,晶澳科技,36784498.000000,电力设备,2010-08-10
125,300750.SZ,宁德时代,8313390.000000,电力设备,2018-06-11
126,300014.SZ,亿纬锂能,681000.000000,电力设备,2009-10-30
127,600885.SH,宏发股份,1066400.000000,电力设备,1996-02-05
128,300724.SZ,捷佳伟创,74506690.000000,电力设备,2018-08-10
129,300207.SZ,欣旺达,6893770.000000,电力设备,2011-04-21
130,600089.SH,特变电工,1631751.000000,电力设备,1997-06-18
131,002623.SZ,亚玛顿,705410.000000,电力设备,2011-10-13


# return calculate

In [6]:
# cbond_list = ['113516.SH','110043.SH',
# ]
cbond_list = idsty_stk_dic[target_industry]  
# ###############################################################
before_trade_date = '20201029'#监测当日前一个交易日
present_date = '20201030' #监测当日
# before_trade_date = '20201026'#监测当日前一个交易日
# present_date = '20201027' #监测当日
# next_trade_date = '20220917' #监测当日下一个自然日
# ###############################################################

start_date = pd.to_datetime(before_trade_date) 

end_date = pd.to_datetime('20221030') #多一个月没关系，切片的时候，不需要这多出来的一个月的对子会切掉

cbond_price = pd.DataFrame([])
for code in cbond_list:
    price_df = w.wsi(code, "close", start_date, end_date,usedf=True)[1]   #只要了收盘价
    price_df.columns = [code]
    cbond_price = pd.concat([cbond_price,price_df],axis=1)
cbond_price = cbond_price.ffill()
# cbond_price = cbond_price[cbond_price.index.time!=time(15,0,0)]
# cbond_price = cbond_price.loc['2022-09-09 15:00:00':] #从前一个交易日的收盘价开始截取
cbond_price_begin_time = before_trade_date[:4]+'-' + \
                    before_trade_date[4:6]+'-' + \
                    before_trade_date[6:8]+' 15:00:00'
cbond_price = cbond_price.loc[cbond_price_begin_time:]
cbond_price

,300769.SZ,300763.SZ,002459.SZ,300750.SZ,300014.SZ,600885.SH,300724.SZ,300207.SZ,600089.SH,002623.SZ,300457.SZ,300037.SZ,002452.SZ,600438.SH,300274.SZ,600458.SH,601012.SH,600525.SH,601615.SH,300316.SZ
2020-10-29 15:00:00,105.00,123.40,40.15,240.00,53.55,50.70,102.61,28.52,8.30,42.65,29.46,74.56,7.12,30.58,32.36,6.79,74.20,6.39,16.07,29.53
2020-10-30 09:30:00,103.49,128.45,40.32,243.98,52.80,50.96,102.42,28.39,8.35,43.10,29.68,74.36,7.11,30.50,36.68,6.84,75.37,6.43,16.79,30.12
2020-10-30 09:31:00,103.11,131.01,39.90,243.90,52.80,50.93,103.33,28.53,8.35,42.70,29.93,74.35,7.10,30.57,37.03,6.80,74.53,6.43,16.68,29.98
2020-10-30 09:32:00,103.12,130.14,39.74,242.14,52.55,50.90,102.96,28.39,8.34,42.81,29.88,74.26,7.09,30.70,36.50,6.81,74.09,6.42,16.64,30.10
2020-10-30 09:33:00,102.82,130.45,39.94,242.98,52.90,50.80,103.64,28.43,8.37,42.80,29.86,73.85,7.10,30.74,38.00,6.83,74.70,6.44,16.80,30.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-28 14:56:00,258.01,193.37,66.73,365.65,83.39,32.96,143.15,23.16,20.74,28.32,19.96,37.45,7.14,45.18,128.92,8.34,49.26,4.79,26.28,74.28
2022-10-28 14:57:00,258.01,193.30,66.75,365.63,83.46,32.96,143.00,23.16,20.74,28.32,19.96,37.45,7.14,45.17,128.93,8.34,49.26,4.78,26.30,74.30
2022-10-28 14:58:00,258.01,193.30,66.75,365.63,83.46,32.96,143.00,23.16,20.74,28.32,19.96,37.45,7.14,45.17,128.93,8.34,49.26,4.78,26.30,74.30
2022-10-28 14:59:00,258.01,193.30,66.75,365.63,83.46,32.96,143.00,23.16,20.74,28.32,19.96,37.45,7.14,45.17,128.93,8.34,49.26,4.78,26.30,74.30


In [7]:
cbond_return = pd.DataFrame([])
# tmp_stk_code = '601006.SH'
# cbond_preclose = cbond_pair_price.iloc[0,0] # 初始时刻的bond前收盘价
stock_preclose = cbond_price.iloc[0,:].tolist() # 初始时刻的stock前收盘价

for index,row in cbond_price.iterrows():
    
    # cbond_return.loc[index,cbond_pair[0]] = row[cbond_pair[0]]/cbond_preclose - 1 # 算日内的return
    for i in range(len(cbond_list)):
        cbond_return.loc[index,cbond_list[i]] = row[cbond_list[i]]/stock_preclose[i] - 1
        if index.time()==time(15,0,0):  #算完日内return，迭代到今日价格的最后一个time bar的时候，将其设为前收盘价
            # cbond_preclose = row[cbond_pair[0]]    # 日内return指标用 现价/昨日收盘价 计算
            stock_preclose[i] = row[cbond_list[i]]
        
# 检查df是否有空值
cbond_return.isnull().sum(axis=0)  # 按列统计
cbond_return.isnull().sum(axis=1) # 按行统计
cbond_return.isnull().sum(axis=1).sum() # 统计整张表

0

In [8]:
cbond_return.isnull().sum(axis=0)  # 按列统计

300769.SZ    0
300763.SZ    0
002459.SZ    0
300750.SZ    0
300014.SZ    0
600885.SH    0
300724.SZ    0
300207.SZ    0
600089.SH    0
002623.SZ    0
300457.SZ    0
300037.SZ    0
002452.SZ    0
600438.SH    0
300274.SZ    0
600458.SH    0
601012.SH    0
600525.SH    0
601615.SH    0
300316.SZ    0
dtype: int64

In [9]:
# cbond_return = cbond_return.dropna()

# correlation & cointegration test

In [10]:
# print(cbond_return.corr(method='pearson').describe())
pd.set_option('display.max_columns',None)
df_corr = cbond_return.corr(method='pearson')
for i in range(len(df_corr)):
    df_corr.iloc[i,i]=np.nan
tmp_se = df_corr.max()
# df_corr
print('max corr code:',f'\'{tmp_se.idxmax()}\'')
tmp_se.sort_values(ascending=False)

max corr code: '300750.SZ'


300750.SZ    0.671295
300014.SZ    0.671295
300763.SZ    0.613816
300274.SZ    0.613816
300207.SZ    0.576327
600438.SH    0.554804
601012.SH    0.554804
600089.SH    0.553459
300724.SZ    0.492088
002459.SZ    0.481224
300316.SZ    0.476557
600885.SH    0.468029
300037.SZ    0.445984
300457.SZ    0.444316
601615.SH    0.424712
300769.SZ    0.399271
600458.SH    0.391983
002623.SZ    0.355220
600525.SH    0.347226
002452.SZ    0.304330
dtype: float64

In [18]:
main_code = '300724.SZ'
tmp_line = df_corr.loc[main_code]
tmp_line = pd.DataFrame(tmp_line)
for index,row in tmp_line.iterrows():
    if row[main_code] == tmp_se.loc[main_code]:
        # print(index)
        sec_code = index
        tmp_corr = tmp_se.loc[main_code]
print(f'\'{main_code}\',\'{sec_code}\'','@',tmp_corr)

'300724.SZ','300274.SZ' @ 0.4920881750756731


In [14]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
def cointegration_test(y, x):
    ols_result = sm.OLS(y, x).fit()
    return adfuller(ols_result.resid)
pair_code = [main_code,sec_code]
x_code = pair_code[0]
y_code = pair_code[1]   
x = cbond_return[x_code].tolist()
y = cbond_return[y_code].tolist()

cointegration_test(y, x),cointegration_test(x, y)

((-18.07430399471006,
  2.598444559258929e-30,
  4,
  117366,
  {'1%': -3.430405718376944,
   '5%': -2.861564626689826,
   '10%': -2.566783107918253},
  -1024834.8936408765),
 (-18.434534152502753,
  2.1659222385356786e-30,
  32,
  117338,
  {'1%': -3.430405731673137,
   '5%': -2.8615646325664894,
   '10%': -2.5667831110462034},
  -1069835.7086634336))

# output to pickle

In [13]:
save_path = f'/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data/price/{target_industry}.pkl'
cbond_price.to_pickle(save_path)
save_path = f'/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data/return/{target_industry}.pkl'
cbond_return.to_pickle(save_path)
target_industry

'电力设备'